In [1]:
import os
from dotenv import load_dotenv

import pandas as pd
from sqlalchemy import create_engine, inspect

In [2]:
load_dotenv()

True

In [3]:
USER = os.getenv('DB_USER')
PASSWORD = os.getenv('DB_PASSWORD')
HOST = os.getenv('DB_HOST')
NAME = os.getenv('DB_NAME')

In [4]:
DATABASE_URI = f"mysql+pymysql://{USER}:{PASSWORD}@{HOST}/{NAME}"

In [5]:
engine = create_engine(DATABASE_URI)

In [6]:
connection = engine.connect()

In [7]:
def select(sql):
    return pd.read_sql(sql, connection)

In [8]:
inspect(engine).get_table_names()

['active_clients',
 'cards',
 'categories',
 'events_data',
 'offers',
 'orders',
 'orders_new']

# Задача: 

Есть таблица логов событий. 
Так вышло, что часть ютм меток при переходах между страницами не прокидывается.
Это создает вопросы от продактов в стиле - "Почему в отчетах много переходов с пустыми ютм-метками?"

Как мы можем решить эту проблему?
(Может, подшаманить данные?)

Или иными словами эта же задача:
Заполнить пустые метки таким образом, чтобы при появлении у юзера события с меткой X всем последующим событиям этого же юзера с пустой меткой присваивалась метка X до появления новой непустой метки Y. После появления метки Y -  всем последующим событиям с пустой меткой присваивалась метка Y и т.д.
Можно обработать только одну метку (например, utm_source). 


In [9]:
sql = '''
      select * from events_data 
      '''
select(sql)

,user_id,event_name,event_timestamp,utm_source,utm_medium,utm_campaign
0,1,first_visit,2023-07-07 13:51:55,,,
1,1,session_start,2023-07-07 13:51:55,,,
2,1,page_view,2023-07-07 13:51:55,vk,cpm,dnp1107
3,1,scroll,2023-07-07 13:52:26,vkontakte,cpm,dnp1107
4,1,form_start,2023-07-07 13:53:32,vkontakte,cpm,dnp1111
5,2,user_engagement,2023-07-07 13:53:48,vkontakte,cpm,dnp1107
6,2,page_view,2023-07-07 13:53:51,,,
7,2,New telegram click,2023-07-07 13:53:51,,,
8,2,click,2023-07-07 13:53:56,,,
9,2,New telegram click,2023-07-07 13:53:57,,,


In [10]:
sql = '''
        WITH cte AS (
            SELECT 
                *,
                ROW_NUMBER() OVER () AS rn
            FROM 
                events_data
        ),
        
        groupses AS (
            SELECT 
                *,
                SUM(CASE WHEN utm_source <> '' THEN 1 ELSE 0 END) OVER (
                    PARTITION BY user_id 
                    ORDER BY rn
                ) AS group_source,
                
                SUM(CASE WHEN utm_medium <> '' THEN 1 ELSE 0 END) OVER (
                    PARTITION BY user_id 
                    ORDER BY rn
                ) AS group_medium,
                
                SUM(CASE WHEN utm_campaign <> '' THEN 1 ELSE 0 END) OVER (
                    PARTITION BY user_id 
                    ORDER BY rn
                ) AS group_campaign
                
            FROM 
                cte
        )
        
        SELECT 
            user_id,
            event_name,
            event_timestamp,
            
            MAX(utm_source) OVER (
                PARTITION BY user_id, group_source
            ) AS utm_source,
            
            MAX(utm_medium) OVER (
                PARTITION BY user_id, group_medium
            ) AS utm_medium,
            
            MAX(utm_campaign) OVER (
                PARTITION BY user_id, group_campaign
            ) AS utm_campaign
            
        FROM 
            groupses
        ORDER BY 
            rn      
      '''
select(sql)

,user_id,event_name,event_timestamp,utm_source,utm_medium,utm_campaign
0,1,first_visit,2023-07-07 13:51:55,,,
1,1,session_start,2023-07-07 13:51:55,,,
2,1,page_view,2023-07-07 13:51:55,vk,cpm,dnp1107
3,1,scroll,2023-07-07 13:52:26,vkontakte,cpm,dnp1107
4,1,form_start,2023-07-07 13:53:32,vkontakte,cpm,dnp1111
5,2,user_engagement,2023-07-07 13:53:48,vkontakte,cpm,dnp1107
6,2,page_view,2023-07-07 13:53:51,vkontakte,cpm,dnp1107
7,2,New telegram click,2023-07-07 13:53:51,vkontakte,cpm,dnp1107
8,2,click,2023-07-07 13:53:56,vkontakte,cpm,dnp1107
9,2,New telegram click,2023-07-07 13:53:57,vkontakte,cpm,dnp1107


In [11]:
connection.close()